In [1]:
#SMA, MA50 값을 채우려면 start_date를 2022년 보다 2~3달 앞으로 땡겨야함

#래그 이펙트를 고려해서 독립변수를 새로 설정하였음, 태훈님이 뽑으신 상위 15개 컬럼에 래그 이펙트 8개를 전부 고려해도 되고 한 개씩 고려해서 비교를 해도 됩니다.


import FinanceDataReader as fdr
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import requests


# 데이터 불러오기 및 전처리
ticker_symbol = "352820"  # 주가 종목 코드 확인, 352820은 하이브의 종목코드
start_date = "2021-07-01"
end_date = "2024-10-24"
stock_data = fdr.DataReader(ticker_symbol, start_date, end_date)
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]

#일별거래상세 및 검색빈도 데이터 추가

volume_search_data = pd.read_csv('volume_plus_search.csv', encoding='utf-8-sig')

# 'datetime'를 인덱스로 설정
volume_search_data['datetime'] = pd.to_datetime(volume_search_data['datetime'])
volume_search_data.set_index('datetime', inplace=True)

# 데이터 병합 (left join)
stock_data = stock_data.join(volume_search_data, how='left')
stock_data = stock_data.drop(axis=1, columns='Unnamed: 0')

# 투자자별 매매동향 데이터 불러오기
trade_data = pd.read_csv("./hive_total_trade.csv")
trade_data['Date'] = pd.to_datetime(trade_data['Date'])

# 투자자별 매매동향 데이터에서 Date를 인덱스로 설정하고, 컬럼으로도 유지
trade_data['Adjusted_Prediction_Date_Copy'] = trade_data['Date']
trade_data.set_index('Date', inplace=True)

trade_data = trade_data.drop(columns=['Adjusted_Prediction_Date_Copy'])

# 투자자별 매매동향 데이터의 컬럼들을 stock_data에 결합
stock_data = stock_data.join(trade_data[['Institution_Total', 'Other_Corporations',	'Individual', 'Foreign_Total']], how='left')


# 이동평균선, RSI, MACD, Bollinger Bands 계산 추가
stock_data['SMA5'] = stock_data['Close'].rolling(window=5).mean()
stock_data['SMA20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()

# RSI 계산 함수
def calculate_RSI(data, window=14):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

stock_data['RSI'] = calculate_RSI(stock_data['Close'], window=14)
stock_data['EMA12'] = stock_data['Close'].ewm(span=12, adjust=False).mean()
stock_data['EMA26'] = stock_data['Close'].ewm(span=26, adjust=False).mean()
stock_data['MACD'] = stock_data['EMA12'] - stock_data['EMA26']
stock_data['Signal_Line'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()
stock_data['Bollinger_Middle'] = stock_data['Close'].rolling(window=20).mean()
stock_data['Bollinger_Upper'] = stock_data['Bollinger_Middle'] + 2 * stock_data['Close'].rolling(window=20).std()
stock_data['Bollinger_Lower'] = stock_data['Bollinger_Middle'] - 2 * stock_data['Close'].rolling(window=20).std()

# Momentum (모멘텀)
stock_data['Momentum'] = stock_data['Close'].diff(periods=5)

# Volume Rate of Change (VROC, 거래량 변화율)
stock_data['VROC'] = stock_data['Volume'].diff(periods=5) / stock_data['Volume'].shift(5) * 100

# Volatility (변동성)
stock_data['Volatility'] = stock_data['Close'].rolling(window=20).std()

# Stochastic Oscillator (%K, %D)
low_min = stock_data['Low'].rolling(window=14).min()
high_max = stock_data['High'].rolling(window=14).max()
stock_data['Stochastic %K'] = 100 * ((stock_data['Close'] - low_min) / (high_max - low_min))
stock_data['Stochastic %D'] = stock_data['Stochastic %K'].rolling(window=3).mean()

# 볼린저 밴드 폭 (BB_width)
stock_data['BB_width'] = stock_data['Bollinger_Upper'] - stock_data['Bollinger_Lower']

# 종가와 시가의 차이 (Close_Open_diff)
stock_data['Close_Open_diff'] = stock_data['Close'] - stock_data['Open']
stock_data.drop(columns=['EMA12', 'EMA26'], inplace=True)

# KOSPI 데이터 (시가와 종가)
kospi_data = fdr.DataReader("KS11", start_date, end_date)[['Open', 'Close']]
kospi_data.rename(columns={'Open': 'KOSPI_Open', 'Close': 'KOSPI_Close'}, inplace=True)

# 나스닥 데이터 (시가와 종가)
nasdaq_data = fdr.DataReader("IXIC", start_date, end_date)[['Open', 'Close']]
nasdaq_data.rename(columns={'Open': 'NASDAQ_Open', 'Close': 'NASDAQ_Close'}, inplace=True)

# 국제 유가 (WTI) 데이터
oil_price_data = fdr.DataReader("CL=F", start_date, end_date)[['Close']]
oil_price_data.rename(columns={'Close': 'WTI_Oil_Price'}, inplace=True)

# KOSPI, 나스닥, WTI 데이터 병합
stock_data = stock_data.join([kospi_data, nasdaq_data, oil_price_data], how='inner')

# 소비자물가지수 데이터 가져오기
cpi_total_index_data = pd.read_csv('소비자물가지수.csv', encoding='utf-8-sig')
cpi_data = cpi_total_index_data[['TIME', 'DATA_VALUE']]
cpi_data['TIME'] = pd.to_datetime(cpi_data['TIME'], format='%Y%m')
cpi_data.set_index('TIME', inplace=True)
cpi_data['DATA_VALUE'] = pd.to_numeric(cpi_data['DATA_VALUE'], errors='coerce')
cpi_daily_df = cpi_data.resample('D').interpolate(method='linear').reset_index()
stock_data = stock_data.join(cpi_daily_df.set_index('TIME')['DATA_VALUE'], how='left')

# DATA_VALUE 열의 이름을 cpi로 변경
stock_data = stock_data.rename(columns={'DATA_VALUE': 'cpi'})

# 이자율 데이터 가져오기
interest_rate_data = pd.read_csv('시장금리.csv', encoding='utf-8-sig')

# datetime을 인덱스로 설정
interest_rate_data['datetime'] = pd.to_datetime(interest_rate_data['datetime'])
interest_rate_data.set_index('datetime', inplace=True)

# 데이터 병합 (left join)
stock_data= stock_data.join(interest_rate_data, how='left')


# 환율 데이터의 datetime을 인덱스로 설정
df_won = pd.read_csv('ExchangeRate.csv', encoding='utf-8-sig')
df_won['datetime'] = pd.to_datetime(df_won['datetime'])
df_won.set_index('datetime', inplace=True)

# 주가 데이터와 환율 데이터 병합 (left join)
stock_data = stock_data.join(df_won, how='left')
stock_data = stock_data.drop(axis=1, columns='Unnamed: 0')

# NaN 값이 있는 행 모두 삭제
stock_data = stock_data.dropna()

# 감성 점수 데이터 불러오기
sentiment_data = pd.read_csv("./hive_news_sentiment_plus_data.csv")
sentiment_data['Adjusted_Prediction_Date'] = pd.to_datetime(sentiment_data['Adjusted_Prediction_Date'])

# 감성 점수 데이터에서 Prediction Date를 인덱스로 설정하고, 컬럼으로도 유지
sentiment_data['Adjusted_Prediction_Date_Copy'] = sentiment_data['Adjusted_Prediction_Date']
sentiment_data.set_index('Adjusted_Prediction_Date', inplace=True)

# 감성 점수 데이터의 'Average_Sentiment_Score'와 'Recommendation'을 stock_data에 결합
stock_data = stock_data.join(sentiment_data[['Average_Sentiment_Score', 'Recommendation', 'Adjusted_Prediction_Date_Copy']], how='left')

# 감성 점수와 추천 데이터가 없는 날은 이전 값으로 채우기 (전방 채우기)
stock_data['Average_Sentiment_Score'] = stock_data['Average_Sentiment_Score'].ffill()
stock_data['Recommendation'] = stock_data['Recommendation'].ffill()


# 감성 점수에 대해 래그 변수를 생성 (1일부터 7일까지)
for i in range(0, 8):
    stock_data[f'Lag_{i}_Sentiment_Score'] = stock_data['Average_Sentiment_Score'].shift(i)


# Target 생성: 다음 날 상승(1) 또는 하락(0)
stock_data['Target'] = np.where(stock_data['Close'].shift(-1) > stock_data['Close'], 1, 0)
stock_data.head(10)

# '' 컬럼을 가장 앞으로 이동
cols = ['Adjusted_Prediction_Date_Copy'] + [col for col in stock_data.columns if col != 'Adjusted_Prediction_Date_Copy']
stock_data = stock_data[cols]

# 결과 확인
print(stock_data.head())


output_path = "processed_stock_data.csv"
stock_data.to_csv(output_path, index=True)
print(f"전처리된 데이터를 '{output_path}' 파일로 저장했습니다.")

           Adjusted_Prediction_Date_Copy    Open    High     Low   Close  \
2022-01-03                    2022-01-03  352000  360500  349000  350500   
2022-01-04                    2022-01-04  350500  351500  336000  337500   
2022-01-05                           NaT  332500  336000  310500  314000   
2022-01-06                    2022-01-06  308000  308500  296500  298500   
2022-01-07                    2022-01-07  302500  307500  300500  302500   

            Volume Pre_market_volume Intraday_volume After_market_volume  \
2022-01-03  179666                 0         178,956                 345   
2022-01-04  223248                 3         220,464                 507   
2022-01-05  500313               135         496,801               1,256   
2022-01-06  459978               417         455,316                 551   
2022-01-07  223108                21         220,405                 470   

           After_hour_volume  ... Recommendation  Lag_0_Sentiment_Score  \
2022-01-03 

C:\Users\USER\AppData\Local\Temp\ipykernel_17628\3337373687.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpi_data['TIME'] = pd.to_datetime(cpi_data['TIME'], format='%Y%m')
C:\Users\USER\AppData\Local\Temp\ipykernel_17628\3337373687.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpi_data['DATA_VALUE'] = pd.to_numeric(cpi_data['DATA_VALUE'], errors='coerce')


In [16]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 665 entries, 2022-01-03 to 2024-10-23
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Adjusted_Prediction_Date_Copy  546 non-null    datetime64[ns]
 1   Open                           665 non-null    int64         
 2   High                           665 non-null    int64         
 3   Low                            665 non-null    int64         
 4   Close                          665 non-null    int64         
 5   Volume                         665 non-null    int64         
 6   Pre_market_volume              665 non-null    object        
 7   Intraday_volume                665 non-null    object        
 8   After_market_volume            665 non-null    object        
 9   After_hour_volume              665 non-null    object        
 10  trading_value                  665 non-null    object        
 11  

In [17]:
stock_data.head(20)

,Adjusted_Prediction_Date_Copy,Open,High,Low,Close,Volume,Pre_market_volume,Intraday_volume,After_market_volume,After_hour_volume,...,Recommendation,Lag_0_Sentiment_Score,Lag_1_Sentiment_Score,Lag_2_Sentiment_Score,Lag_3_Sentiment_Score,Lag_4_Sentiment_Score,Lag_5_Sentiment_Score,Lag_6_Sentiment_Score,Lag_7_Sentiment_Score,Target
2022-01-03,2022-01-03,352000,360500,349000,350500,179666,0,"178,956",345,365,...,1.0,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-01-04,2022-01-04,350500,351500,336000,337500,223248,3,"220,464",507,"2,274",...,1.0,2.000000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-01-05,NaT,332500,336000,310500,314000,500313,135,"496,801","1,256","2,121",...,1.0,2.000000,2.000000,0.500000,NaN,NaN,NaN,NaN,NaN,0
2022-01-06,2022-01-06,308000,308500,296500,298500,459978,417,"455,316",551,"3,694",...,1.0,2.571429,2.000000,2.000000,0.500000,NaN,NaN,NaN,NaN,1
2022-01-07,2022-01-07,302500,307500,300500,302500,223108,21,"220,405",470,"2,212",...,1.0,2.000000,2.571429,2.000000,2.000000,0.500000,NaN,NaN,NaN,0
2022-01-10,2022-01-10,302000,302000,289000,295500,275890,140,"273,897",291,"1,562",...,1.0,3.500000,2.000000,2.571429,2.000000,2.000000,0.500000,NaN,NaN,0
2022-01-11,NaT,294000,300500,288000,289000,209051,137,"206,678",601,"1,635",...,1.0,3.500000,3.500000,2.000000,2.571429,2.000000,2.000000,0.500000,NaN,1
2022-01-12,2022-01-12,295500,302500,293000,297000,223419,3,"221,327",200,"1,889",...,1.0,2.250000,3.500000,3.500000,2.000000,2.571429,2.000000,2.000000,0.500000,0
2022-01-13,2022-01-13,298500,299000,288000,291500,179105,1,"178,101",401,602,...,1.0,2.000000,2.250000,3.500000,3.500000,2.000000,2.571429,2.000000,2.000000,0
2022-01-14,2022-01-14,284500,285000,276500,284000,282689,9,"280,275",919,"1,486",...,1.0,2.000000,2.000000,2.250000,3.500000,3.500000,2.000000,2.571429,2.000000,0


In [18]:
stock_data.tail(20)

,Adjusted_Prediction_Date_Copy,Open,High,Low,Close,Volume,Pre_market_volume,Intraday_volume,After_market_volume,After_hour_volume,...,Recommendation,Lag_0_Sentiment_Score,Lag_1_Sentiment_Score,Lag_2_Sentiment_Score,Lag_3_Sentiment_Score,Lag_4_Sentiment_Score,Lag_5_Sentiment_Score,Lag_6_Sentiment_Score,Lag_7_Sentiment_Score,Target
2024-09-23,2024-09-23,160900,160900,157700,158000,169540,30,"166,747",762,"2,001",...,0.0,-1.375000,-1.000000,2.000000,-0.333333,-0.846154,2.000000,2.000000,4.000000,1
2024-09-24,2024-09-24,158800,163300,157700,162900,201866,0,"199,436",904,"1,526",...,0.0,-0.666667,-1.375000,-1.000000,2.000000,-0.333333,-0.846154,2.000000,2.000000,0
2024-09-25,2024-09-25,164200,164700,159600,159700,182649,0,"179,720","1,030","1,899",...,0.0,-0.833333,-0.666667,-1.375000,-1.000000,2.000000,-0.333333,-0.846154,2.000000,1
2024-09-26,2024-09-26,161000,172000,161000,169900,359860,368,"352,655","5,797","1,040",...,1.0,0.750000,-0.833333,-0.666667,-1.375000,-1.000000,2.000000,-0.333333,-0.846154,1
2024-09-27,2024-09-27,169900,174000,167500,172900,205833,530,"204,404",76,823,...,1.0,0.333333,0.750000,-0.833333,-0.666667,-1.375000,-1.000000,2.000000,-0.333333,0
2024-09-30,2024-09-30,173900,175000,168700,168800,97366,11,"96,117",329,909,...,0.0,0.000000,0.333333,0.750000,-0.833333,-0.666667,-1.375000,-1.000000,2.000000,1
2024-10-02,NaT,165500,171600,165500,170100,101129,1,"100,492",167,469,...,0.0,0.000000,0.000000,0.333333,0.750000,-0.833333,-0.666667,-1.375000,-1.000000,1
2024-10-04,2024-10-04,169200,174000,169100,173500,103045,14,"102,506",238,287,...,1.0,2.666667,0.000000,0.000000,0.333333,0.750000,-0.833333,-0.666667,-1.375000,0
2024-10-07,2024-10-07,172800,175600,169000,173500,117563,118,"116,870",116,459,...,1.0,0.200000,2.666667,0.000000,0.000000,0.333333,0.750000,-0.833333,-0.666667,1
2024-10-08,2024-10-08,172200,179600,172100,178800,207172,0,"206,533",186,453,...,0.0,-0.428571,0.200000,2.666667,0.000000,0.000000,0.333333,0.750000,-0.833333,0
